In [3]:
!pip install matplotlib
!pip install tensorflow
!pip install tensorflow_datasets

  Obtaining dependency information for wrapt>=1.11.0 from https://files.pythonhosted.org/packages/19/2b/548d23362e3002ebbfaefe649b833fa43f6ca37ac3e95472130c4b69e0b4/wrapt-1.16.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow_datasets from https://files.pythonhosted.org/packages/fe/18/4865973f5469cfe33bbe1cfc2f1918335eb44f4cc3d316c1bce22c1af2bc/tensorflow_datasets-4.9.4-py3-none-any.whl.metadata
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/101.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/101.3 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/101.3 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/101.3 kB ? eta -:--:--
     --------

ERROR: Could not install packages due to an OSError: [WinError 5] Adgang nægtet: 'C:\\Users\\khavu\\anaconda3\\envs\\amlfall23\\Lib\\site-packages\\google\\~rotobuf\\internal\\_api_implementation.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [ ]:
# This function 'convert_samples' takes a sample from a dataset, which is assumed to have
# a dictionary-like structure with keys 'image' and 'label'. The purpose of this function
# is to preprocess the image and label data before it is used in a machine leraning model
def convert_sample(sample):
    # Extract the 'image' and 'label' from the sample
    image, label = sample['image'], sample['label']

    # Convert the image to a floating-point format (tf.float32)
    image = tf.image.convert_image_dtype(image, tf.float32)

    # Convert the label to one-hot encoding with 2 classes (assuming binary classification)
    label = tf.one_hot(label, 2, dtype=tf.float32)

    # Return the preprocessed image and label
    return image, label

In [ ]:
# Loading the PCam dataset using Tensorflow datasets(tfds)
ds1,ds2,ds3 = tfds.load('patch_camelyon',split=['train[:20%]','test[:5%]','validation[:5%]'],
                        data_dir = '/content/drive/MyDrive',
                        download=True,
                        shuffle_files=True)

In [ ]:
# Mapping and batching datasets using the convert_sample function
train_dataset       = ds1.map(convert_sample).batch(32)
validation_dataset  = ds3.map(convert_sample).batch(32)
test_dataset        = ds2.map(convert_sample).batch(32)

In [ ]:
def first_ccn_model():
    input_img = Input(shape=(96,96,3))

    x = Conv2D(16, (3, 3), padding='valid', activation='relu')(input_img)
    x = Conv2D(32, (3, 3), padding='valid', activation='relu')(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    y = Dense(2, activation='softmax')(x)

    model = Model(inputs=input_img, outputs=y)
    return model

# Optimizer: SGD is chosen as the optimizer with specific parameters, including a learning rate of 0.01,
# momentum of 0.9, and Nesterov momentum enabled
sgd_opt = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)


model = first_ccn_model()

# Model Compilation: the model is compiled using the specified optimizer(sgd_opt),
# categorical crossentropy as the loss function, and accuracy as the metric to monitor during training
model.compile(optimizer=sgd_opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training: the model is trained using the 'fit' method with training data and validation.
# The training is set to run for 2 epochs
hist = model.fit(train_dataset,
                 validation_data=validation_dataset,
                 epochs=2)